In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [30]:
def calc(c, max_iter):
    i = 0
    while i < max_iter:
        c = c**2 + c
        if np.abs(c) > 2:
            return i
    return i

In [38]:
(complex(3, 4)) ** 2

(-7+24j)